In [7]:
import sys
sys.path.append('..\\')
import os
import re
import numpy as np
import gensim
import nltk
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
from scripts.epoch_logger import EpochLogger
from scripts.predict_from_context import predict_output_word

## Training

In [8]:
DATA_FOLDER = "..\\data\\"
train_path = os.path.join(DATA_FOLDER, "train.txt")
with open(train_path) as file:
    text = file.read()

In [9]:
data = [nltk.word_tokenize(re.sub(r"[^a-z]+", " ", sentence.lower())) for sentence in nltk.sent_tokenize(text)]

### FastText

In [16]:
epoch_logger = EpochLogger()
fast_text_model = FastText(sg=1, min_n=2, max_n=3, workers=12, callbacks=(epoch_logger), size=100)

# build the vocabulary
fast_text_model.build_vocab(data)

In [18]:
# train the model
fast_text_model.train(data, total_examples=fast_text_model.corpus_count, epochs=10, callbacks=(epoch_logger,))

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end


In [26]:
fast_text_model.save('models\\fasttext\\fasttext200k.model')

### Word2Vec

In [12]:
word2vec_model = Word2Vec(size=300, workers=12)
word2vec_model.build_vocab(data)

In [20]:
word2vec_model.train(data, total_examples=model.corpus_count, epochs=20, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end


(744806832, 973533000)

In [21]:
word2vec_model.save("models\\word2vec200k\\word2vec200k.model")

## Testing

In [108]:
test_sent = "The level of postoperative pain and nausea was scored in the postanesthesia care unit ( PACU ) during the first postoperative hour , as well as at 2 , 4 , 8 and 24h postoperatively ."
words = nltk.word_tokenize(re.sub(r"[^a-z]+", " ", test_sent.lower()))
words

['the',
 'level',
 'of',
 'postoperative',
 'pain',
 'and',
 'nausea',
 'was',
 'scored',
 'in',
 'the',
 'postanesthesia',
 'care',
 'unit',
 'pacu',
 'during',
 'the',
 'first',
 'postoperative',
 'hour',
 'as',
 'well',
 'as',
 'at',
 'and',
 'h',
 'postoperatively']

In [101]:
fast_text_model = FastText.load("models\\fasttext200k\\fasttext200k.model")
word2vec_model = Word2Vec.load("models\\word2vec200k\\word2vec200k.model")

In [116]:
def check(words):
    print(words)
    return (predict_output_word(word2vec_model, words[:-1]), predict_output_word(fast_text_model, words[:-1]))

In [117]:
check(words[10:15])

['the', 'postanesthesia', 'care', 'unit', 'pacu']


([('unit', 0.695053),
  ('pacu', 0.057000123),
  ('postanesthesia', 0.03609574),
  ('care', 0.014875768),
  ('arrival', 0.008621463),
  ('stay', 0.0078104693),
  ('intensive', 0.007062314),
  ('postanaesthesia', 0.005120231),
  ('ward', 0.0020113364),
  ('postanesthetic', 0.0015265796)],
 [('intensive', 0.6312368),
  ('postanesthesia', 0.15752365),
  ('unit', 0.103116766),
  ('usual', 0.06519048),
  ('pacu', 0.00407928),
  ('inpatient', 0.002983677),
  ('orthogeriatric', 0.0024374798),
  ('maternity', 0.0023734183),
  ('cpu', 0.0020048202),
  ('stay', 0.0018834673)])